Scrape Wikipedia table and store it in the dataframe.

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]

Correct the header by setting the first row of the dataframe to be the header.

In [2]:
# Create a new variable called 'header' from the first row of the dataset
header = df.iloc[0]
# Rename the dataframe's column values with the header variable
df.columns = header
# Replace the dataframe with a new one which does not contain the first row
df = df[1:]

Remove rows where Borough is Not assigned.

In [3]:
df = df[df.Borough != 'Not assigned']

Group by Postcode and Borough. Concatenate multiple Neighborhoods. Reset index.

In [4]:
df['Neighbourhood'] = df[['Postcode','Borough','Neighbourhood']].groupby(['Postcode','Borough'])['Neighbourhood'].transform(','.join)
df.drop_duplicates(subset =['Postcode','Borough','Neighbourhood'], inplace=True)
df.reset_index(inplace=True, drop=True)

If a cell has a borough but a Not assigned neighborhood, set the neighborhood to be the same as the borough.

In [5]:
df['Neighbourhood'] = df.apply(
    lambda row: row['Borough'] if row['Neighbourhood']=='Not assigned' else row['Neighbourhood'],
    axis=1
)

In [6]:
df.shape

(103, 3)